# 🖖🏼 Mindfulness AI (mai)    

A novel, optimally small language model builder for generating a semantically tuned taxonomy of medical content.   
Prompted with a natural English input (statement or query), a content (repository) and (explainable taxonomy model) are emitted/generated from (PubMed) content and (UMLS) concepts.   
The subject of "mindfulness" is generated in this example.   
The resulting Mindfulness AI is implemented with Algolia search, (here).   

### query and collect from pubmed

### query and collect from UMLS

### normalize vectors and generate JSON

### generate SVG

### notes ...

[[{Value1:1},{value2:2}],[{value3:3},{value4:4}],.....]   
<= for data like that use this =>   
value1 = df['column_name'][0][0].get(Value1)   


In [6]:
import pandas as pd
# from urllib.request import urlopen

# pd.read_json("sample.json",lines=True,orient='columns')
df_data = pd.read_json('data.json')

newdf = df_data.drop("qualified",axis='columns')

print(newdf)

df_data.head()

# https://realpython.com/urllib-request/
from urllib.request import urlopen
response = None
try:
    response = urlopen("https://www.example.com")
except Exception as ex:
    print(ex)
else:
    body = response.read()
finally:
    if response is not None:
        response.close()

# with urlopen("https://www.example.com") as response:
#     body = response.read()


  provenance                                                url  \
0     pubmed  https://pubmed.ncbi.nlm.nih.gov/28031068/?form...   
1     pubmed  https://pubmed.ncbi.nlm.nih.gov/35634214/?form...   
2     github                                             github   
3   nlm umls                                           nlm umls   

                                               title                bonus  
0  What defines mindfulness-based programs? The w...                  NaN  
1  Meditators' Non-academic Definition of Mindful...                  NaN  
2                                                NaN                  NaN  
3                                                NaN  {'name': 'library'}  
